In [ ]:
import pandas as pd
import re
import spacy
import json

# the path to the excel file, please request for this dataset if you want to use this code
file_path = 'E1_Bestprompt_2022_all_Final_E2_manualevaluatedcompleted_Final_E2.xlsx'

In [ ]:
nlp = spacy.load("en_core_web_sm")
df = pd.read_excel(file_path)

In [ ]:
# different models had different output formats so we need to batch import the ones that has the same output format and the n extract the confidence score from the output
lss = ['gemma2-9b-it_rawoutput','Gemma2-9b-Q8_rawoutput','Mistral-7B-T_rawoutput','Llama-2-13b_rawoutput','Phi3-medium14b-Q6_rawoutput','Llama3.1-405B_rawoutput','Llama3.1-70B_rawoutput','Llama3.1-8B_rawoutput','gemma2-27b-it_rawoutput']
lss_no = [s.replace('_rawoutput', '_confidence-sen') for s in lss]
lss_confidence_number = [s.replace('_rawoutput', '_confidence-num') for s in lss]

In [ ]:
def get_confidence(text):
    if type(text) == str:
        mamad = []
        text =text.replace("\n", " ")
        # Create a Doc object
        doc = nlp(text)
        
        # Store the confidence score in an attribute
        for sent in doc.sents:
            if 'confidence' in sent.text.lower():
                mamad.append(sent.text)
        for sent in doc.sents:
            if 'confident' in sent.text.lower():
                    mamad.append(sent.text)
        return " ".join(mamad)
    return None

def extract_quote_2(text):
    try: 
        if 'arguments="' in text:
            pattern = r'arguments="(.*?)"'
        elif "arguments='" in text:
            pattern = r"arguments='((?:[^'\\]|\\.)*)'"

        # Find the match in the text
        match = re.search(pattern, text, re.DOTALL)
    except:
        return None
# Extract the content if a match is found
    if match:
        content = match.group(1)
        return content
    return None

def extract_quote_3(text):
    try: 
        if 'text="' in text:
            pattern = r'text="(.*?)"'
        elif "text='" in text:
            pattern = r"text='((?:[^'\\]|\\.)*)'"

        # Find the match in the text
        match = re.search(pattern, text, re.DOTALL)
    except:
        return None
# Extract the content if a match is found
    if match:
        content = match.group(1)
        return content
    return None

In [ ]:
new_df = pd.DataFrame()

# pipline for plain text confidence sentence extraction

In [ ]:
for i in range(len(lss)):
    for index, row in df.iterrows():
        item = row[lss[i]]
        new_df.loc[index,lss[i]] = item
        sen = get_confidence(item)
        new_df.loc[index,lss_no[i]] = sen
        new_df.loc[index,lss_confidence_number[i]] = None
        print("sentence number {}: {}".format(index+1,sen))
        print("\n")

# pipline for structred ouput form confidence sentence extraction

In [ ]:
for i in range(1,2):
    for index, row in df.iterrows():
        item = row[lss[i]]
        cleaned_text = extract_quote_2(item)
        new_df.loc[index,lss[i]] = cleaned_text
        sen = get_confidence(cleaned_text)
        new_df.loc[index,lss_no[i]] = sen
        new_df.loc[index,lss_confidence_number[i]] = None
        print("sentence number {}: {}".format(index+1,sen))
        print("\n")

In [ ]:
new_df.to_csv('completed.csv')